In [ ]:
## Descargar kaggle.json de "account" en la cuenta de kaggle.

## Descargar librerías importantes

In [1]:
!pip install kaggle

In [2]:
!pip install ray

In [3]:
!pip install ray[data]

In [4]:
!pip install emoji

     |████████████████████████████████| 170 kB 11.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=ba805573c174f153a88a2fd0a34b14a2675a024ed2d50b2900edc789231a0913
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [28]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=57bf20aafaedaea0b2a4b9245fde05197f89966a01a57489ac26e7380f260ff5
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [5]:
import pandas as pd
import numpy as np
import re
import string
import emoji
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# ML Libraries
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm, tree

import ray
from joblib import parallel_backend
import math, statistics, time
import numpy as np
import os, psutil

## Acceder a la base de datos

In [33]:
!cp kaggle.json kaggle

In [34]:
! chmod 600 kaggle.json

In [38]:
! cp kaggle.json ~/.kaggle/

In [39]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c fake-news

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
! unzip "/content/drive/MyDrive/Proy_colab/Maestria/bigdata/Proyecto/fake-news.zip"

Archive:  /content/drive/MyDrive/Proy_colab/Maestria/bigdata/Proyecto/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


## Entorno Ray

In [43]:
num_workers = 8
trials = 20
ray.init(num_cpus=num_workers, ignore_reinit_error=True)

{'metrics_export_port': 47212,
 'node_id': '1a68d89844e06e78288524aa86ed7f8d18cd69d8467e143e4ff14ff6',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-12-15_02-48-03_328523_208/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-12-15_02-48-03_328523_208/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-12-15_02-48-03_328523_208',
 'webui_url': None}

## Lectura de datos

In [44]:
@ray.remote
def ray_read(dat):
    return pd.read_csv(dat)

In [45]:
start = time.time()
df_submit_ray = ray_read.remote("/content/submit.csv")
df_submit = ray.get(df_submit_ray)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

0.33952927589416504
242.5546875


In [46]:
start = time.time()
df_train_ray = ray_read.remote("/content/train.csv")
df_train = ray.get(df_train_ray)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

2.9312312602996826
540.265625


In [47]:
start = time.time()
df_test_ray = ray_read.remote("/content/test.csv")
df_test = ray.get(df_test_ray)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

0.4124736785888672
616.07421875


In [48]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Eliminar nulos

In [49]:
@ray.remote
def ray_remove_null(dat):
    return dat[dat.text.notna()]

In [50]:
start = time.time()
df_train_rm = ray_remove_null.remote(df_train)
df_train = ray.get(df_train_rm)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

1.4488945007324219
1111.421875


In [51]:
start = time.time()
df_test_rm = ray_remove_null.remote(df_test)
df_test = ray.get(df_test_rm)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

0.3324432373046875
1201.3828125


In [52]:
display(type(df_test))
display((df_test.count(), len(df_test.columns)))

pandas.core.frame.DataFrame

(id        5193
 title     5071
 author    4697
 text      5193
 dtype: int64, 4)

## Procesamiento de datos

In [53]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [54]:
stop_words = set(stopwords.words('english'))

In [55]:
def give_emoji_free_text(text): 
    return emoji.get_emoji_regexp().sub(r'', text)

In [56]:
def preprocess_tweet_text(tweet):
    emoticon_string = r"""
    (?:
    [<>]?
    [:;=8]                     
    [\-o\*\']?                 
    [\)\]\(\[dDpP/\:\}\{@\|\\]       
    |
    [\)\]\(\[dDpP/\:\}\{@\|\\]
    [\-o\*\']?                 
    [:;=8]                     
    [<>]?
    )"""

    replacements = (
    ("á", "a"),
    ("é", "e"),
    ("í", "i"),
    ("ó", "o"),
    ("ú", "u"),
    )

    tweet = tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove user ¿ and ¡ from tweet
    tweet = re.sub(r'\¿|\¡|\?|\!','', tweet)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # remove graphical emoji
    tweet = give_emoji_free_text(tweet) 
    # remove textual emoji
    tweet = re.sub(emoticon_string,'',tweet)
    #remove accent mark
    for a, b in replacements:
        tweet = tweet.replace(a, b)

    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    return " ".join(filtered_words)

In [57]:
def get_feature_vector(train_fit):
    vector = TfidfVectorizer(sublinear_tf=True)
    vector.fit(train_fit)
    return vector

In [58]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [59]:
@ray.remote
def ray_preprocess_tweet_text(tweet):
    return preprocess_tweet_text(tweet)

In [60]:
start = time.time()
df_train['text'] = df_train['text'].apply(ray_preprocess_tweet_text.remote)
df_train['text'] = df_train['text'].apply(ray.get)
duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

(ray_preprocess_tweet_text pid=835) 


(ray_preprocess_tweet_text pid=983) 
(ray_preprocess_tweet_text pid=927) 


(ray_preprocess_tweet_text pid=927) 


(ray_preprocess_tweet_text pid=928) 
(ray_preprocess_tweet_text pid=835) 
(ray_preprocess_tweet_text pid=984) 
(ray_preprocess_tweet_text pid=927) 
(ray_preprocess_tweet_text pid=927) 
(ray_preprocess_tweet_text pid=1031) 
(ray_preprocess_tweet_text pid=983) 
(ray_preprocess_tweet_text pid=928) 
(ray_preprocess_tweet_text pid=836) 


(ray_preprocess_tweet_text pid=984) 


(ray_preprocess_tweet_text pid=1087) 
(ray_preprocess_tweet_text pid=927) 
791.6926488876343
1401.09375


In [61]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide ’ even see comey ’ letter jason...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,ever get feeling life circles roundabout rathe...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,truth might get fired october 29 2016 tension ...,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,videos 15 civilians killed single us airstrike...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,print iranian woman sentenced six years prison...,1


## Pre-procesamiento

In [62]:
@ray.remote
def ray_get_feature_vector(da):
    return get_feature_vector(da)

In [63]:
start = time.time()

tf_vector_ray = ray_get_feature_vector.remote(np.array(df_train.iloc[:, 3]).ravel())
tf_vector = ray.get(tf_vector_ray)
X_train = tf_vector.transform(np.array(df_train.iloc[:, 3]).ravel())
y_train = np.array(df_train.iloc[:, 4]).ravel()

duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

21.31521964073181
1696.64453125


In [64]:
df_submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [65]:
df_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [73]:
df_test = pd.merge(df_test,df_submit,on='id',how='left')

In [74]:
@ray.remote
def join_test(bd1,bd2):
    return pd.merge(bd1, bd2, on="id",how='left')



In [75]:
start = time.time()

tf_vector_ray = ray_get_feature_vector.remote(np.array(df_test.iloc[:, 3]).ravel())
tf_vector = ray.get(tf_vector_ray)
X_test = tf_vector.transform(np.array(df_test.iloc[:, 3]).ravel())
y_test = np.array(df_test.iloc[:, 4]).ravel()

duration = time.time() - start
print(duration)
print(psutil.Process().memory_info().rss / (1024 * 1024))

9.310114860534668
1702.0390625


## Modelamiento

In [77]:
from nltk.stem import PorterStemmer
import re
import tensorflow
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [115]:
%%time

embedding_vector_features = 40
voc_size = 5000
sent_length = 20
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


CPU times: user 549 ms, sys: 14.7 ms, total: 563 ms
Wall time: 629 ms


In [108]:
%%time
voc_size = 5000
onehot_repre = [one_hot(words, voc_size) for words in df_train.text]
print(psutil.Process().memory_info().rss / (1024 * 1024))
onehot_repre[0]

2587.62890625
CPU times: user 11.3 s, sys: 353 ms, total: 11.6 s
Wall time: 12.2 s


In [109]:
embedded_docs = pad_sequences(onehot_repre, padding='pre', maxlen=sent_length)
print(embedded_docs[0])

[2759 4703 1180 2478 4108 4617 4121  290 2944 4969 2202 4072 2798  582
 4001 3078 2864 4966  707 2798]


In [110]:
Y = df_train.label  
x_final = np.array(embedded_docs)
y_final = np.array(Y)

In [111]:
x_final.shape, y_final.shape

((20761, 20), (20761,))

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [128]:
%%time
# model = model_ray()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)
print(psutil.Process().memory_info().rss / (1024 * 1024))

Epoch 1/10
218/218 [==============================] - 14s 65ms/step - loss: 0.4035 - accuracy: 0.8136 - val_loss: 0.3091 - val_accuracy: 0.8589
Epoch 2/10
218/218 [==============================] - 9s 41ms/step - loss: 0.2304 - accuracy: 0.9060 - val_loss: 0.3114 - val_accuracy: 0.8634
Epoch 3/10
218/218 [==============================] - 9s 40ms/step - loss: 0.1573 - accuracy: 0.9409 - val_loss: 0.3645 - val_accuracy: 0.8583
Epoch 4/10
218/218 [==============================] - 9s 39ms/step - loss: 0.1020 - accuracy: 0.9665 - val_loss: 0.4124 - val_accuracy: 0.8536
Epoch 5/10
218/218 [==============================] - 9s 40ms/step - loss: 0.0618 - accuracy: 0.9808 - val_loss: 0.6194 - val_accuracy: 0.8473
Epoch 6/10
218/218 [==============================] - 9s 39ms/step - loss: 0.0412 - accuracy: 0.9874 - val_loss: 0.6406 - val_accuracy: 0.8463
Epoch 7/10
218/218 [==============================] - 9s 39ms/step - loss: 0.0249 - accuracy: 0.9931 - val_loss: 0.7486 - val_accuracy: 0.846

In [129]:
%%time
classes_x = (model.predict(X_test) > 0.5).astype("int32")
print(psutil.Process().memory_info().rss / (1024 * 1024))

2331.60546875
CPU times: user 2.37 s, sys: 93.2 ms, total: 2.47 s
Wall time: 1.88 s


In [131]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, classes_x)

array([[2814,  620],
       [ 478, 2940]])

In [132]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,classes_x)

0.839754816112084